In [1]:
import sys; sys.path.append('../..')
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random
import pyzx as zx
import os
import pickle
import time
from fractions import Fraction

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [2]:
random.seed(1344)
# g = zx.generate.cliffordT(qubits=8, depth=1000, p_t=0.4)
# c = zx.Circuit.from_graph(g)
c = zx.Circuit.load('../../evaluation/feyn_bench/before/mod5_4_before').to_basic_gates().split_phase_gates()
print(c.stats())

Circuit mod5_4_before on 5 qubits with 63 gates.
        28 is the T-count
        35 Cliffords among which 
        28 2-qubit gates (28 CNOT, 0 other) and
        6 Hadamard gates.


In [3]:
c.to_graph().to_json()

'{"wire_vertices": {"b0": {"annotation": {"boundary": true, "coord": [0, 0], "input": true, "output": false}}, "b1": {"annotation": {"boundary": true, "coord": [0, -1], "input": true, "output": false}}, "b2": {"annotation": {"boundary": true, "coord": [0, -2], "input": true, "output": false}}, "b3": {"annotation": {"boundary": true, "coord": [0, -3], "input": true, "output": false}}, "b4": {"annotation": {"boundary": true, "coord": [0, -4], "input": true, "output": false}}, "b5": {"annotation": {"boundary": true, "coord": [49, 0], "input": false, "output": true}}, "b6": {"annotation": {"boundary": true, "coord": [49, -1], "input": false, "output": true}}, "b7": {"annotation": {"boundary": true, "coord": [49, -2], "input": false, "output": true}}, "b8": {"annotation": {"boundary": true, "coord": [49, -3], "input": false, "output": true}}, "b9": {"annotation": {"boundary": true, "coord": [49, -4], "input": false, "output": true}}}, "node_vertices": {"v0": {"annotation": {"coord": [1, -4]

In [3]:
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
zx.Circuit.from_graph(g_tele).split_phase_gates().stats()

'Circuit  on 24 qubits with 727 gates.\n        173 is the T-count\n        554 Cliffords among which \n        385 2-qubit gates (350 CNOT, 35 other) and\n        80 Hadamard gates.'

In [18]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
zx.extract_circuit(g_full.copy()).stats()

'Circuit  on 24 qubits with 963 gates.\n        173 is the T-count\n        790 Cliffords among which \n        437 2-qubit gates (152 CNOT, 285 other) and\n        344 Hadamard gates.'

In [24]:
g_greedy = g_tele.copy()
zx.simplify.greedy_simp(g_greedy)
zx.extract_circuit(g_greedy.copy()).stats()

'Circuit  on 24 qubits with 1021 gates.\n        173 is the T-count\n        848 Cliffords among which \n        335 2-qubit gates (3 CNOT, 332 other) and\n        444 Hadamard gates.'

In [4]:
g_nu = g_tele.copy()
g_nu.normalize()
zx.simplify.greedy_simp_neighbors(g_nu,cap=0,max_v=True)
zx.extract_circuit(g_nu.copy()).stats()

apply pivot match  (4, (819, 861), None, None)
apply pivot match  (3, (707, 757), None, None)
apply pivot match  (3, (924, 948), None, None)
apply pivot match  (3, (359, 437), 354, None)
apply pivot match  (2, (306, 313), None, None)
apply pivot match  (2, (147, 149), None, None)
apply pivot match  (2, (574, 621), None, None)
apply pivot match  (3, (571, 666), None, None)
apply pivot match  (3, (634, 639), None, None)
apply pivot match  (2, (261, 263), None, None)
apply pivot match  (2, (173, 680), None, None)
apply pivot match  (2, (1072, 1074), None, None)
apply pivot match  (2, (603, 608), None, None)
apply pivot match  (2, (171, 682), None, None)
apply pivot match  (2, (1023, 1069), 1019, None)
apply pivot match  (2, (738, 773), 731, None)
apply pivot match  (2, (204, 206), None, None)
apply pivot match  (2, (652, 683), 645, None)
apply pivot match  (2, (985, 990), None, None)
apply pivot match  (2, (136, 258), None, None)
apply pivot match  (2, (380, 385), None, None)
apply pivot 

apply pivot match  (0, (527, 1373), 525, None)
apply pivot match  (0, (525, 1374), None, 1375)
apply pivot match  (0, (519, 521), None, 522)
apply pivot match  (0, (522, 1378), None, 1379)
apply pivot match  (0, (142, 144), None, 145)
apply pivot match  (0, (601, 1332), None, 1333)
apply pivot match  (0, (83, 85), None, 1284)
apply pivot match  (0, (92, 152), 1305, None)
apply pivot match  (0, (524, 1377), 1381, None)
apply pivot match  (0, (554, 591), None, 592)
apply pivot match  (0, (176, 178), None, 1353)
apply pivot match  (0, (292, 1283), 1355, None)
apply pivot match  (0, (209, 211), 207, None)
apply pivot match  (0, (242, 244), None, 330)
apply lcomp match  (1.0, (1400, [330, 1401]), None)
apply lcomp match  (1.0, (1401, [124, 330]), None)
apply pivot match  (0, (124, 1308), None, 1309)
apply pivot match  (0, (99, 1336), None, 1337)
apply pivot match  (0, (341, 343), None, 435)
apply pivot match  (0, (223, 225), None, 731)
apply lcomp match  (1.0, (1406, [731, 1407]), None)
app

'Circuit  on 24 qubits with 932 gates.\n        173 is the T-count\n        759 Cliffords among which \n        306 2-qubit gates (3 CNOT, 303 other) and\n        411 Hadamard gates.'

In [5]:
g_test = g_tele.copy()
zx.draw(g_test, labels=True)

In [4]:
gc = g_tele.copy()
gc.normalize()
zx.simplify.spider_simp(gc)
zx.simplify.to_gh(gc)
zx.simplify.id_simp(gc)
zx.simplify.spider_simp(gc)

spider_simp: 13. 6. 3. 3. 2. 1.  6 iterations
id_simp: 5.  1 iterations
spider_simp: 4.  1 iterations


1

In [5]:
zx.heuristics.neighbor_unfusion.lcomp_matcher(gc, zx.gflow.gflow(gc)[1])

found some neighbors for vertex  13 [15, 11]
found some neighbors for vertex  15 [13]
found some neighbors for vertex  16 [21]
found some neighbors for vertex  22 [24, 8]
found some neighbors for vertex  24 [25, 22]
found some neighbors for vertex  25 [24]
found some neighbors for vertex  27 [40]
found some neighbors for vertex  28 [33, 21]
found some neighbors for vertex  33 [28]
found some neighbors for vertex  34 [39]
found some neighbors for vertex  39 [47, 34]
found some neighbors for vertex  40 [42, 27]
found some neighbors for vertex  42 [43, 40]
found some neighbors for vertex  43 [42]
found some neighbors for vertex  47 [49, 39]
found some neighbors for vertex  50 [52, 49]
found some neighbors for vertex  52 [50]
found some neighbors for vertex  53 [58]
found some neighbors for vertex  58 [66, 53]
found some neighbors for vertex  59 [61, 7]
found some neighbors for vertex  61 [62, 59]
found some neighbors for vertex  62 [61]
found some neighbors for vertex  66 [68, 58]
found s

[(1.0, (11, [9, 13]), None),
 (-3.0, (13, [15, 5, 11]), 15),
 (-3.0, (13, [15, 5, 11]), 11),
 (-2.0, (15, [13, 16]), 13),
 (-5.0, (16, [15, 8, 21, 5]), 21),
 (1.0, (21, [28, 16]), None),
 (-3.0, (22, [24, 8, 5]), 24),
 (-3.0, (22, [24, 8, 5]), 8),
 (-2.0, (24, [22, 25]), 25),
 (-2.0, (24, [22, 25]), 22),
 (-3.0, (25, [24, 27, 5]), 24),
 (-5.0, (27, [25, 28, 34, 40]), 40),
 (-5.0, (28, [21, 27, 33, 7]), 33),
 (-5.0, (28, [21, 27, 33, 7]), 21),
 (-2.0, (33, [34, 28]), 28),
 (-5.0, (34, [33, 27, 39, 7]), 39),
 (-3.0, (39, [47, 46, 34]), 47),
 (-3.0, (39, [47, 46, 34]), 34),
 (-3.0, (40, [42, 27, 7]), 42),
 (-3.0, (40, [42, 27, 7]), 27),
 (-2.0, (42, [40, 43]), 43),
 (-2.0, (42, [40, 43]), 40),
 (-3.0, (43, [42, 46, 7]), 42),
 (-3.0, (47, [49, 39, 7]), 49),
 (-3.0, (47, [49, 39, 7]), 39),
 (1.0, (49, [47, 50]), None),
 (-3.0, (50, [49, 52, 6]), 52),
 (-3.0, (50, [49, 52, 6]), 49),
 (-2.0, (52, [50, 53]), 50),
 (-5.0, (53, [52, 7, 58, 6]), 58),
 (-3.0, (58, [66, 65, 53]), 66),
 (-3.0, (58, 

In [7]:
gf = zx.gflow.gflow(gc)
gff= zx.heuristics.gflow_calculation.focus_gflow(gc,gf)
for v in gc.non_outputs():
    nset = zx.heuristics.neighbor_unfusion.get_possible_unfusion_neighbors_3(gc,v,None,gf)
    if nset:
        print("found some neighbors for vertex ",v,nset)

found some neighbors for vertex  0 {5}
found some neighbors for vertex  1 {6}
found some neighbors for vertex  2 {7}
found some neighbors for vertex  3 {8}
found some neighbors for vertex  4 {9}
found some neighbors for vertex  5 {0, 69, 75, 13, 78, 81, 86, 25}
found some neighbors for vertex  6 {1, 66, 50, 62}
found some neighbors for vertex  7 {2, 43, 47, 28}
found some neighbors for vertex  8 {3, 9}
found some neighbors for vertex  9 {4}
found some neighbors for vertex  11 {9, 13}
found some neighbors for vertex  13 {11}
found some neighbors for vertex  15 {16, 13}
found some neighbors for vertex  16 {21, 8}
found some neighbors for vertex  21 {16, 28}
found some neighbors for vertex  22 {24, 8, 5}
found some neighbors for vertex  24 {25, 22}
found some neighbors for vertex  25 {24, 27, 5}
found some neighbors for vertex  27 {25, 28}
found some neighbors for vertex  28 {21}
found some neighbors for vertex  33 {34, 28}
found some neighbors for vertex  34 {39}
found some neighbors for

In [8]:
zx.draw(gc, labels=True)

In [11]:
from fractions import Fraction
zx.heuristics.neighbor_unfusion.unfuse_to_neighbor(gc, 13, 15, Fraction(1,2))

> /home/korbinian/Code/pyzx-heuristics/pyzx/heuristics/neighbor_unfusion.py(150)unfuse_to_neighbor()
    148     import pdb
    149     pdb.set_trace()
--> 150     new_phase = split_phases(g.phases()[vertex], desired_phase)
    151     phase_spider = g.add_vertex(VertexType.Z,-2,g.rows()[vertex],new_phase)
    152     g.set_phase(vertex, desired_phase)

ipdb> c
unfuse to neighbor  13 15 92 91
> /home/korbinian/Code/pyzx-heuristics/pyzx/heuristics/neighbor_unfusion.py(160)unfuse_to_neighbor()
    158     print("unfuse to neighbor ",vertex, neighbor, phaseless_spider, phase_spider)
    159     pdb.set_trace()
--> 160     return (phaseless_spider, phase_spider)
    161 
    162 def apply_lcomp(g: BaseGraph[VT,ET], match, gfl):

ipdb> c


(92, 91)

In [14]:
gfnew = zx.gflow.gflow(gc)

In [18]:
for v in [13,15,91,92]:
    print(str(v),gfnew[1][v])

13 {92}
15 {16, 22}
91 {15}
92 {91}


In [31]:
gc = g_tele.copy()
gc.normalize()
zx.simplify.spider_simp(gc)
zx.simplify.to_gh(gc)
zx.simplify.id_simp(gc)
zx.simplify.spider_simp(gc)

spider_simp: 13. 6. 3. 3. 2. 1.  6 iterations
id_simp: 5.  1 iterations
spider_simp: 4.  1 iterations


1

In [5]:
gold = zx.gflow.gflow(gc)

In [35]:
for v in [13,11,91,92]:
    print(str(v),gfnew[1][v])

TypeError: 'NoneType' object is not subscriptable

In [32]:
def get_broken_matches():
    broken_matches = []
    good_match = None
    for match in zx.heuristics.neighbor_unfusion.lcomp_matcher(gc, zx.gflow.gflow(gc)[1]):
        if match[2]: #is neighbor unfusion match
            orig_phase = gc.phase(match[1][0])
            phaseless_s, phase_s = zx.heuristics.neighbor_unfusion.unfuse_to_neighbor(gc, match[1][0], match[2], Fraction(1,2))
            if not zx.gflow.gflow(gc):
    #             print("broken gflow from match ",match[1][0], match[2])
                broken_matches.append((match[1][0], match[2]))
            else:
                if not good_match:
                    good_match = match
            gc.remove_vertex(phaseless_s)
            gc.remove_vertex(phase_s)
            gc.add_edge(gc.edge(match[1][0], match[2]), 2) #revert all
            gc.set_phase(match[1][0], orig_phase)
    return broken_matches, good_match

In [28]:
zx.draw(gc, labels=True, scale=30)

In [33]:
bm1, good_match = get_broken_matches()
zx.heuristics.neighbor_unfusion.apply_lcomp(gc, good_match, zx.gflow.gflow(gc)[1])
bm2, good_match2 = get_broken_matches()

apply lcomp match  (-3.0, (13, [15, 5, 11]), 15)


In [36]:
zx.draw(gc, labels=True, scale=30)

In [44]:
fggraph = zx.heuristics.gflow_calculation.build_focused_gflow_graph(gc, zx.gflow.gflow(gc))

In [45]:
zx.draw(fggraph)

In [23]:
gex = gc.copy()
zx.simplify.full_reduce(gex)
cex = zx.extract_circuit(gex.copy())

In [25]:
cex.stats()

'Circuit  on 5 qubits with 64 gates.\n        8 is the T-count\n        56 Cliffords among which \n        29 2-qubit gates (0 CNOT, 29 other) and\n        24 Hadamard gates.'